In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install transformers
!pip install tensorflow-addons
import tensorflow_addons as tfa
# !pip install datasets
# from datasets import *
# from datasets import load_dataset

!pip install -q -U tensorflow-text
import tensorflow_text as text
import functools

In [3]:
import tensorflow as tf
def auto_select_accelerator():
    TPU_DETECTED = False
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.TPUStrategy(tpu)
        print("Running on TPU:", tpu.master())
        TPU_DETECTED = True
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")

    return strategy, TPU_DETECTED
tf.keras.backend.clear_session()
strategy, tpu_detected = auto_select_accelerator()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.125.75.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.125.75.114:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Running on TPU: grpc://10.125.75.114:8470
Running on 8 replicas


In [4]:
!cd gig

import os
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

import tensorflow as tf
from tensorflow.keras.models import Model
import tensorflow.keras.layers as L
from sklearn.model_selection import train_test_split
base_dir = '/content/drive/MyDrive/Colab Notebooks/keytohead/'
data_dir = base_dir + 'data/'
SAVE_DIR = base_dir + 'models/'
GCS_PATH = 'gs://stylex/'

from utils import *
from models import *
from dataloader import *
from tokenizer_configs import tokenizer, bert_tokenizer, random_selector, _VOCAB, _PLAIN_TOKEN, _HUMOR_TOKEN, _ROMANCE_TOKEN, _CLICK_TOKEN, _MASK_TOKEN
from configs import *
from layers import *
from translator import *

In [6]:
clickbait = pd.read_csv(data_dir+'clickbait/raw/clickbait.raw', header = None, names = ['text'])
humor = pd.read_csv(data_dir+'humor/raw/humor.raw', sep = '이런건없겠지', header = None, names = ['text'], engine='python')
romance = pd.read_csv(data_dir+'romance/raw/romance.raw', sep = '이런건없겠지', header = None, names = ['text'], engine='python')
headlines_train = pd.read_csv(data_dir+'CNN_NYT/raw/headlines.train', sep = '이런건없겠지', header = None, names = ['text'], engine='python')
headlines_valid = pd.read_csv(data_dir+'CNN_NYT/raw/headlines.valid', sep = '이런건없겠지', header = None, names = ['text'],engine='python')
headlines_test = pd.read_csv(data_dir+'CNN_NYT/raw/headlines.test', sep = '이런건없겠지', header = None, names = ['text'],engine='python')

In [7]:
clickbait_train, clickbait_valid = train_test_split(clickbait, test_size = 0.2, shuffle = True)
clickbait_valid, clickbait_test = train_test_split(clickbait_valid, test_size = 0.5, shuffle = True)
humor_train, humor_valid = train_test_split(humor, test_size = 0.2, shuffle = True)
humor_valid, humor_test = train_test_split(humor_valid, test_size = 0.5, shuffle = True)
romance_train, romance_valid = train_test_split(romance, test_size = 0.2, shuffle = True)
romance_valid, romance_test = train_test_split(romance_valid, test_size = 0.5, shuffle = True)

In [8]:
TRAIN_SIZE = len(romance_train) + len(humor_train) + len(clickbait_train) + len(headlines_train)
TRAIN_SIZE

1341134

In [9]:
# BERT의 단어 집합을 vocabulary.txt에 저장
with open('vocabulary.txt', 'w') as f:
  for token in tokenizer.vocab.keys():
    f.write(token + '\n')
df = pd.read_fwf('vocabulary.txt', header=None)
df

,0
0,[PAD]
1,[unused1]
2,[unused2]
3,[unused3]
4,[unused4]
...,...
28991,##）
28992,##，
28993,##－
28994,##／


In [10]:
hds_process = lambda x : tf_bert_sampler(x, style_token = _PLAIN_TOKEN)#'random')
hds_train = get_ds_pd(headlines_train['text'], repeat = True, shuffle=True, process = hds_process)
hds_valid = get_ds_pd(headlines_valid['text'], process = hds_process)
hds_test = get_ds_pd(headlines_test['text'], process = hds_process)

humor_process = lambda x : tf_bert_sampler(x, style_token = _HUMOR_TOKEN)
humords_train = get_ds_pd(humor_train['text'], repeat = True, shuffle=True, process = humor_process)
humords_valid = get_ds_pd(humor_valid['text'], process = humor_process)
humords_test = get_ds_pd(humor_test['text'], process = humor_process)

romance_process = lambda x : tf_bert_sampler(x, style_token = _ROMANCE_TOKEN)
romanceds_train = get_ds_pd(romance_train['text'], repeat = True, shuffle=True, process = romance_process)
romanceds_valid = get_ds_pd(romance_valid['text'], process = romance_process)
romanceds_test = get_ds_pd(romance_test['text'], process = romance_process)

click_process = lambda x : tf_bert_sampler(x, style_token = _CLICK_TOKEN)
clickds_train = get_ds_pd(clickbait_train['text'], repeat = True, shuffle=True, process = click_process)
clickds_valid = get_ds_pd(clickbait_valid['text'], process = click_process)
clickds_test = get_ds_pd(clickbait_test['text'], process = click_process)

In [11]:
train_dss = [hds_train, humords_train, romanceds_train, clickds_train]
valid_dss = [hds_valid, humords_valid, romanceds_valid, clickds_valid]
test_dss = [hds_test, humords_test, romanceds_test, clickds_test]
train_ds = tf.data.Dataset.sample_from_datasets(train_dss, weights=[0.25, 0.25, 0.25, 0.25])
valid_ds = tf.data.Dataset.sample_from_datasets(valid_dss, weights=[0.25, 0.25, 0.25, 0.25])
test_ds = tf.data.Dataset.sample_from_datasets(test_dss, weights=[0.25, 0.25, 0.25, 0.25])

In [12]:
for x in hds_test:
    temp = x
    print(x)
    break

((<tf.Tensor: shape=(512, 64), dtype=int64, numpy=
array([[    1,   101,  1285, ...,     0,     0,     0],
       [    1,   101,  7893, ...,     0,     0,     0],
       [    1,   101,  3343, ...,     0,     0,     0],
       ...,
       [    1,   101,  4856, ...,     0,     0,     0],
       [    1,   101,  1106, ...,     0,     0,     0],
       [    1,   101, 20086, ...,     0,     0,     0]])>, <tf.Tensor: shape=(512, 64), dtype=int64, numpy=
array([[  101,  1567,  2995, ...,     0,     0,     0],
       [  101,  7893,  1141, ...,     0,     0,     0],
       [  101,   187, 13356, ...,     0,     0,     0],
       ...,
       [  101,   170, 16753, ...,     0,     0,     0],
       [  101,  1184,  1106, ...,     0,     0,     0],
       [  101,   189,  1183, ...,     0,     0,     0]])>), <tf.Tensor: shape=(512, 64), dtype=int64, numpy=
array([[ 1567,  2995,   131, ...,     0,     0,     0],
       [ 7893,  1141,  8362, ...,     0,     0,     0],
       [  187, 13356,  1811, ...,   

In [13]:
with strategy.scope():
    model = AttConv1D_BE_12()

    loss = loss_function
    count = TRAIN_SIZE // BATCH_SIZE // 4
    lr_decayed_fn = tf.keras.experimental.CosineDecay(1e-3, count)
    adamw = tfa.optimizers.AdamW(lr_decayed_fn, learning_rate=1e-4)
    mac = accuracy_function
    model.compile(loss = loss, optimizer = adamw, metrics = [mac])

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 64)]         0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 128)          0           ['input_1[0][0]',                
                                                                  'input_2[0][0]']                
                                                                                                  
 tf.math.equal (TFOpLambda)     (None, 128)          0           ['concatenate[0][0]']        

In [15]:
 model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    SAVE_DIR + 'style_conformer_random_L12_768_fin.h5', monitor="val_accuracy_function", verbose=1, save_best_only=True,
    save_weights_only=True, mode="max", save_freq="epoch"
)

In [16]:
model.fit(train_ds, epochs=100, steps_per_epoch = TRAIN_SIZE // BATCH_SIZE, validation_data = valid_ds, callbacks = [model_ckpt])

Epoch 1/100
 192/2619 [=>............................] - ETA: 9:16 - loss: 7.3083 - accuracy_function: 0.2527

In [ ]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/HG/models/style_conformer_random_L12_768_fin.h5')

In [ ]:
translator = Translator(tokenizer, model)
tester_ds = get_ds_pd(headlines_test['text'], process = hds_process, batch_size = 1, shuffle =True)

In [1]:
!pip install -q tf-models-official==2.4.0
import os

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

import os
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow_text as text  # A dependency of the preprocessing model
import tensorflow_addons as tfa
from official.nlp import optimization
import numpy as np

tf.get_logger().setLevel('ERROR')
os.environ["TFHUB_MODEL_LOAD_FORMAT"]="UNCOMPRESSED"

In [2]:
gs_folder_bert = "gs://cloud-tpu-checkpoints/bert/cased_L-12_H-768_A-12/"#tensorflow/bert_en_cased_L-12_H-768_A-12
tf.io.gfile.listdir(gs_folder_bert)

In [ ]:
import json

bert_config_file = os.path.join(gs_folder_bert, "bert_config.json")
config_dict = json.loads(tf.io.gfile.GFile(bert_config_file).read())

bert_config = bert.configs.BertConfig.from_dict(config_dict)

config_dict

In [ ]:
with strategy.scope():
  bert_classifier, bert_encoder = bert.bert_models.classifier_model(
    bert_config, num_labels=2)

In [ ]:
bert_encoder.layers[20]